# Gridding

Here we present multiple solutions and approaches on the gridding problem for a potentially huge dataset.

The organization of the source codes are as follow:  

## `python/` 

Contains the treatment of the problem in pure python, the approaches taken on each of the versions are as follow:
   - `v0_original.py`: The original version of the code.
   - `v1_index_calc_jitted.py`: Calculating the grid indices has been Just In Compiled (JIT).
   - `v2_gridding_jitted.py`: The whole gridding function has been compiled.
   - `v3_single_timestep_vectorized.py`: Calculation of a single timestep of the grid has been fully vectorized using numpy intrinsic functions.
   - `v4_single_timestep_vectorized_jitted.py`: On top of vectorizing a single timestep the function for calculating a single timestep of the grid has been compiled.
   - `v5_gridding_vectorized.py`: The whole gridding function has been fully vectorized using numpy.
   - `v6_gridding_vectorized_multithreaded.py`: On top of vectorizing the gridding function it uses python `concurrent` library to parallelize the gridding over `n_workers` threads using chunks of the dataset.
   - `v7_mpi_timesteps.py`: Using `mpi4py` library we divide the computation of the grid over timsteps to multiple processes.
   - `v8_mpi_baselines.py`: Same as above but divide the dataset over baseline pairs wrather than timesteps to multiple process.

### Benchmarking

Here we present the benchmarking results obtained on the pure python implementation. Note that these benchmarks are done on a single node consisting of an **